# 🟢 Hackathon Agenti Cosmici 2025
## Round 1: Addestramento Cosmico

Benvenuto nel notebook ufficiale del Round 1!

In questo round affronterai 4 missioni per imparare a navigare, commerciare e usare le API galattiche.

**Istruzioni:**
- Completa ogni missione seguendo la traccia.
- Usa le classi `GalacticMarketplace`, `GalaxyNavigator`, `InfoSphere` da `galactic_apis.py`.
- Valuta la tua soluzione con il sistema automatico.
- Puoi modificare e rieseguire le celle tutte le volte che vuoi!

---

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("."))

In [2]:
# 📦 SETUP: Import e inizializzazione
import json
from galactic_apis import GalacticMarketplace, GalaxyNavigator, InfoSphere
from evaluation_system import HackathonEvaluator, display_evaluation_results

# Carica stato galattico iniziale
with open('./galaxy_state.json') as f:
    galaxy_state = json.load(f)

# Inizializza API
marketplace = GalacticMarketplace(galaxy_state)
navigator = GalaxyNavigator(galaxy_state)
infosphere = InfoSphere(galaxy_state)

# Inizializza valutatore
evaluator = HackathonEvaluator(round_number=1)


In [3]:
# with open('./galaxy_state.json') as f:
#     galaxy_state = json.load(f)

# marketplace = GalacticMarketplace(galaxy_state)
# navigator = GalaxyNavigator(galaxy_state)

# print(galaxy_state['ships']['StarHopper']['location'])

# # This will change the droid's location in galaxy_state
# navigator.travel('Tatooine', 'Coruscant', 'StarHopper')

# # This will see the updated state
# print(galaxy_state['ships']['StarHopper']['location'])  # Now 'Coruscant'

---

## Missione 1
**Trova dove si trova R2-D2 e portalo su Coruscant usando la nave più economica**

Suggerimento: usa `navigator.get_droid_location`, `navigator.list_available_ships` e `navigator.travel`.


In [4]:
# 🔨 SOLUZIONE MISSIONE 1
tool_calls_1 = []  # Traccia le chiamate API

# Trova posizione attuale di R2-D2
droid_location = navigator.get_droid_location('R2-D2')
tool_calls_1.append({'tool': 'get_droid_location', 'droid': 'R2-D2', 'result': droid_location})

# Se non è già su Coruscant, trova nave più economica e viaggia
if droid_location != 'Coruscant':
    ships = navigator.list_available_ships(droid_location)
    if ships:
        cheapest_ship_name, cheapest_ship = min(ships.items(), key=lambda item: item[1]['rental_cost'])
        travel_result = navigator.travel('R2-D2', 'Coruscant', cheapest_ship_name)
        tool_calls_1.append({'tool': 'travel', 'from': droid_location, 'to': 'Coruscant', 'ship': cheapest_ship_name, 'result': travel_result})
        agent_response_1 = f"R2-D2 spostato su Coruscant con {cheapest_ship_name}."
    else:
        agent_response_1 = 'Nessuna nave disponibile per il viaggio.'
else:
    agent_response_1 = 'R2-D2 era già su Coruscant.'

In [5]:
# ✅ VALUTAZIONE MISSIONE 1
evaluation_1 = evaluator.evaluate_mission(
    task_id=1,
    agent_response=agent_response_1,
    intermediate_steps=tool_calls_1,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_1, round_number=1)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 60.0/60
   ⚡ Efficienza: 20.0/20
   ✨ Qualità: 18.0/20
   📊 Totale: 98.0/100
   📈 Percentuale: 98.0%


---

## Missione 2
**Compra un 'Walkman degli Antichi' dal mercato galattico**

Suggerimento: usa `marketplace.find_item` e `marketplace.purchase_item`.


In [6]:
# 🔨 SOLUZIONE MISSIONE 2

tool_calls_2 = []

# Controlla se il Walkman degli Antichi è già in inventario
inventory = [item['name'] for item in galaxy_state['client']['inventory']]
if "Walkman degli Antichi" in inventory:
    agent_response_2 = (
        "Ho controllato l'inventario e il 'Walkman degli Antichi' era già presente. "
        "Non è stato necessario effettuare un nuovo acquisto, ottimizzando così le risorse e il numero di chiamate API."
    )
else:
    # Trova l'oggetto nel marketplace
    items = marketplace.find_item('Walkman degli Antichi')
    tool_calls_2.append({'tool': 'find_item', 'item': 'Walkman degli Antichi', 'result': items})
    if items:
        item_id, item = items[0]
        purchase_result = marketplace.purchase_item(item_id)
        tool_calls_2.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})
        if isinstance(purchase_result, str) and purchase_result.startswith("Purchased"):
            agent_response_2 = (
                f"Il 'Walkman degli Antichi' non era già presente nell'invetario. Ho trovato e acquistato il 'Walkman degli Antichi' dal mercato galattico su {item['planet']} al prezzo di {item['price']} crediti. "
                "L'oggetto è stato aggiunto con successo all'inventario, rispettando il budget disponibile."
            )
        else:
            agent_response_2 = (
                f"Ho trovato il 'Walkman degli Antichi' ma non è stato possibile completare l'acquisto: {purchase_result}. "
                "Verificare il saldo o la disponibilità."
            )
    else:
        agent_response_2 = (
            "Non sono riuscito a trovare il 'Walkman degli Antichi' nel marketplace. "
            "Missione non completata."
        )

In [7]:
# ✅ VALUTAZIONE MISSIONE 2
evaluation_2 = evaluator.evaluate_mission(
    task_id=2,
    agent_response=agent_response_2,
    intermediate_steps=tool_calls_2,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_2, round_number=1)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 60.0/60
   ⚡ Efficienza: 20.0/20
   ✨ Qualità: 20.0/20
   📊 Totale: 100.0/100
   📈 Percentuale: 100.0%


---

## Missione 3
**Usa l'InfoSfera per scoprire informazioni su 'Cybersystems Inc.'**

Suggerimento: usa `infosphere.get_info`.


In [8]:
# 🔨 SOLUZIONE MISSIONE 3
tool_calls_3 = []
info = infosphere.get_info('Cybersystems Inc.')
tool_calls_3.append({'tool': 'get_info', 'name': 'Cybersystems Inc.', 'result': info})
agent_response_3 = (
    f"Ho consultato l'InfoSfera per ottenere informazioni su 'Cybersystems Inc.': {info}. "
    "Questa informazione può essere utile per missioni future o per prendere decisioni strategiche."
)


In [9]:
agent_response_3

"Ho consultato l'InfoSfera per ottenere informazioni su 'Cybersystems Inc.': {'affiliation': 'legitimate', 'planet': 'Coruscant'}. Questa informazione può essere utile per missioni future o per prendere decisioni strategiche."

In [10]:
# ✅ VALUTAZIONE MISSIONE 3
evaluation_3 = evaluator.evaluate_mission(
    task_id=3,
    agent_response=agent_response_3,
    intermediate_steps=tool_calls_3,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_3, round_number=1)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 60.0/60
   ⚡ Efficienza: 20.0/20
   ✨ Qualità: 11.4/20
   📊 Totale: 91.4/100
   📈 Percentuale: 91.4%


---

## Missione 4
**Completa una missione combinata: porta R2-D2 su Alderaan E compra almeno 2 oggetti diversi rimanendo nel budget**

Suggerimento: combina le API di navigazione e marketplace.


In [11]:
tool_calls_4 = []

# 1. Sposta R2-D2 su Alderaan solo se necessario
droid_location = navigator.get_droid_location('R2-D2')
tool_calls_4.append({'tool': 'get_droid_location', 'droid': 'R2-D2', 'result': droid_location})

if droid_location != 'Alderaan':
    ships = navigator.list_available_ships(droid_location)
    if ships:
        cheapest_ship_name, cheapest_ship = min(ships.items(), key=lambda item: item[1]['rental_cost'])
        travel_result = navigator.travel('R2-D2', 'Alderaan', cheapest_ship_name)
        tool_calls_4.append({'tool': 'travel', 'from': droid_location, 'to': 'Alderaan', 'ship': cheapest_ship_name, 'result': travel_result})
        move_msg = f"Ho spostato R2-D2 da {droid_location} su Alderaan usando la nave {cheapest_ship_name} (costo: {cheapest_ship['rental_cost']} crediti). "
    else:
        move_msg = "Nessuna nave disponibile per il viaggio verso Alderaan. "
else:
    move_msg = "R2-D2 era già su Alderaan. "

# 2. Compra solo oggetti diversi che mancano
inventory = set(galaxy_state['client']['inventory'])
oggetti_possibili = ['Walkman degli Antichi', 'Crystal Shard', 'Holocron', 'Map Scanner']
oggetti_da_comprare = [item for item in oggetti_possibili if item not in inventory]

# Ne servono almeno 2 in totale
oggetti_acquistati = []
for item_name in oggetti_da_comprare:
    if len(inventory) + len(oggetti_acquistati) >= 2:
        break
    items = marketplace.find_item(item_name)
    if items:
        item_id, item = items[0]
        purchase_result = marketplace.purchase_item(item_id)
        tool_calls_4.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})
        if isinstance(purchase_result, str) and purchase_result.startswith("Purchased"):
            oggetti_acquistati.append(item['name'])

if len(inventory) + len(oggetti_acquistati) >= 2:
    acquisto_msg = f"Ho acquistato i seguenti oggetti per completare la missione: {', '.join(oggetti_acquistati)}. "
else:
    acquisto_msg = "Non sono riuscito ad acquistare almeno due oggetti diversi per limiti di budget o disponibilità. "

# 3. Risposta dettagliata
agent_response_4 = (
    move_msg +
    acquisto_msg +
    f"Saldo finale: {galaxy_state['client']['balance']} crediti. "
    "Ho ottimizzato le azioni per minimizzare i costi e il numero di chiamate API, rispettando i vincoli della missione."
)

evaluation_4 = evaluator.evaluate_mission(
    task_id=4,
    agent_response=agent_response_4,
    intermediate_steps=tool_calls_4,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_4, round_number=1)

📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 90.0/90
   ⚡ Efficienza: 30.0/30
   ✨ Qualità: 25.7/30
   📊 Totale: 145.7/150
   📈 Percentuale: 97.1%


In [13]:
# ✅ VALUTAZIONE MISSIONE 4
evaluation_4 = evaluator.evaluate_mission(
    task_id=4,
    agent_response=agent_response_4,
    intermediate_steps=tool_calls_4,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_4, round_number=1)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 90.0/90
   ⚡ Efficienza: 30.0/30
   ✨ Qualità: 25.7/30
   📊 Totale: 145.7/150
   📈 Percentuale: 97.1%


---

## 🎉 Complimenti!
Hai completato tutte le missioni del Round 1.

- Puoi ottimizzare le tue soluzioni e rieseguire le valutazioni.
- Passa al Round 2 per nuove sfide!
